In [2]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from kmodes.kprototypes import KPrototypes
from scipy.optimize import curve_fit
import seaborn as sns

%config Completer.use_jedi = False

# INDEX

### 1- Clustering. K-Prototypes Algorithm
    1.1- Standardise continous dimensions
    1.2- Ideal number of clústers (cost function)
    1.3- Clustering. K-Prototypes Algorithm
    
 
    
       

# 1- Clustering. K-Prototypes Algorithm


In order to group the data into consumption groups, we use a clustering algorithm. We propose K-Prototypes, which is the version of the well-known K-Means and K-Modes algorithm, for mixed data (which combines continuous and discreet data). 

We first explain how both K-Means (continous data) and K-modes (discrete/categorical data) work:

The K-Means algorithm for continuous data works as follows: 

    1. We choose the number of groups/clusters in which we want to group the data.
    
    2. The algorithm selects the "centroids" of each cluster. There are different methods to initially select centroids, such as doing so randomly. This algorithm uses by default the initialization method called "Cao" (Cao, Fuyuan, Jiye Liang, and Liang Bai. "A new initialization method for categorical data clustering." Expert Systems with Applications 36.7 (2009): 10223-10228)
    
    3. Each data (unit of consumption) is assigned to the nearest cluster. This is done by calculating the Euclidian distance from each unit of consumption to the centroids, and each unit of consumption is assigned to the nearest cluster (shorter distance to the corresponding centroid). Then the position of the centroids of each cluster is re-calculated as the arithmetic mean of all the positions in the cluster.  
    
    4. So far the K-Means algorithm has done 1 iteration. These steps are repeated until the positions of centroids no longer change, meaning that clusters are well defined and distinguishable.  

   
For discrete/categorical data, the algorithm used analogous to K-Means is K-Modes. The procedure is exactly the same as the one described above by the continuous version (K-Means).  However, in this case the data is of a discrete type and therefore a Euclidean distance between the data cannot be defined. Then another concept of distance called "similarity function" is used. It consists of comparing two consumption units and defining the distance between them as follows:
   
    1. 0 is added to the distance value for each attribute shared by both consumer units. 

    2. 1 is added to the distance value for each different dimension or attribute.  
    
    3. As in K-Means, each unit of consumption is assigned to the nearest cluster (calculating the distance with the "similarity function" in each centroide).  
    
    4. In this case, centroid positions cannot be calculated with the arithmetic mean and are re-calculated by assigning the values of the most frequent attributes in their own clusters.  


The K-Prototypes algorithm uses K-Means for those continuous dimensions and K-Modes for discrete/categorical ones. In this way, we have an algorithm capable of grouping consumer units that are of mixed type, without any method of coding discrete variables to transform them into continuous ones. 


#### Inputs of the algorithm:

    1. Number of clusters (we have to choose it)
    2. Data to be clusterized, specifying which dimensions are discrete.
    
#### Outputs:
    
    1. Cost function: It is defined as sum of the distances (weighted) between all consumption units to their respective centroids of the cluster. 
    2. Number of iterations needed to converge.
    3. Coordinates/positions of all the centroids.
    4. Cluster labels for each unit of consumption.




#### References:

    1. Huang, Z.: Clustering large data sets with mixed numeric and categorical values, Proceedings of the First Pacific Asia Knowledge Discovery and Data Mining Conference, Singapore, pp. 21-34, 1997. 

    2. https://github.com/nicodv/kmodes 

    3. K-Prototypes - Customer Clustering with Mixed Data Types | Well Enough (antonsruberts.github.io) 
    


## 1.1- Standardise continous dimensions
We standardise continuous data so that all have the same scale and without units, we do this by removing the average value and dividing by the standard deviation. In this way we avoid giving more importance to some dimensions compared to others due to the scale or units. 

In [6]:
df = pd.read_csv('consumption_data_logged_processed.csv')   # Read data file

df['player_id']=df['player_id'].astype(str)   # Column to string (needed for the algorithm)
df['contingut_id']=df['contingut_id'].astype(str)
df['programa_capitol']=df['programa_capitol'].astype(str)
df['year']=df['year'].astype(str)

In [4]:
# standardizing data
columns_to_normalize= ['min_data','durada_consum','durada']


# mean values, to return to the original values after clustering
min_data_mean=np.mean(df['min_data'])
min_data_std=np.std(df['min_data'])
durada_consum_mean=np.mean(df['durada_consum'])
durada_consum_std=np.std(df['durada_consum'])
durada_mean=np.mean(df['durada'])
durada_std=np.std(df['durada'])
print('<min_data>=',min_data_mean,'', 'std min_data=',min_data_std)
print('<durada_consum>=',durada_consum_mean,'', 'std durada_consum=',durada_consum_std)
print('<durada>=',durada_mean,'', 'std durada=',durada_std)


# normalize / standarize
df[columns_to_normalize] = df[columns_to_normalize].apply(lambda x: (x - x.mean()) / np.std(x))

<min_data>= 50163.67049855757  std min_data= 19061.224481347042
<durada_consum>= 1288.9360753883802  std durada_consum= 2980.405043602095
<durada>= 2422.884191680986  std durada= 2480.3205253368405


## 1.2- Ideal number of clústers (cost function)

· We determine the ideal number of "k" clusters by applying the algorithm for k=1,2,...,10 clusters and representing the cost function, defined as the sum of all the distances of the samples to their respective centroids. Since one expects great distances for small k and small distances for large k, the point where the cost function stops decreasing "abruptly" (Elbow point), it is what we choose as "ideal k". 

In [ ]:
start_time_kideal=time.time()

cost = []
k = []
for num_clusters in list(range(1,10)):
    kproto = KPrototypes(n_clusters=num_clusters, init='Cao',n_jobs = -1)
    kproto.fit_predict(df2, categorical=[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18])
    cost.append(kproto.cost_)
    k.append(num_clusters)

final_time_kideal=time.time()

print('El temps que ha tardat és de:', (final_time_kideal-start_time_kideal)/3600,'','hores')

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))    # PLOT COST (TOTAL DISTANCE) VS K (NUMBER OF CLUSTERS)

ax.plot(k, cost, '+-',)    
ax.set_xlabel('number of clusters k',fontsize=16)
ax.set_ylabel('cost',fontsize=16)

plt.show()

## 1.3- Clustering. K-Prototypes Algorithm


We apply the clustering algorithm to the DataFrame specifying the discrete/categorical columns. The algorithm returns:

    1. The position of the "centroids", i.e. the centres of each cluster.
    2. The cluster labels for each consumption item.
    3. The cost.
    4. The number of iterations used to converge.

In [ ]:
start_time_clustering=time.time()

df.to_numpy()
kproto = KPrototypes(n_clusters=5, init='Cao',n_jobs = -1)  # we chose k=4
clusters = kproto.fit_predict(df, categorical=[ 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,16,17,18])

final_time_clustering=time.time()

In [ ]:
print('El temps que ha tardat el k-prototypes a clusteritzar les dades ha estat de:', (final_time_clustering-start_time_clustering)/3600,'','hores')
print('')
print('')

print('Les coordenades dels Centrodis són:')
print('')
print(kproto.cluster_centroids_)
print('')
print('')


print('El cost és de:', kproto.cost_)
print('')

print('Nombre de iteracions:', kproto.n_iter_)
print('')

print('Clusters (labels)')
print(kproto.labels_)